In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import deepchem as dc
from rdkit import Chem
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader as PyGDataLoader

In [3]:
from torch_geometric.data import InMemoryDataset, Data

class DeepChemToPyGDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        """
        Converts a list of PyG `Data` objects into an InMemoryDataset.
        
        Args:
            data_list (list of Data): List of PyG Data objects.
            transform (callable, optional): Optional transform applied to each data object.
        """
        super(DeepChemToPyGDataset, self).__init__('.', transform, None, None)
        self.data, self.slices = self.collate(data_list)

    def get(self, idx):
        """
        Retrieves the data object at the specified index.
        """
        return super().get(idx)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, global_mean_pool
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, global_mean_pool

class ARGA(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim, dropout=0.2):
        super(ARGA, self).__init__()
        self.encoder = Encoder(node_features, edge_features, hidden_dim, latent_dim, dropout)
        self.decoder = Decoder(latent_dim, hidden_dim, node_features, edge_features)
        self.discriminator = Discriminator(latent_dim, hidden_dim)
        
    def encode(self, x, edge_index, edge_attr, batch=None):
        return self.encoder(x, edge_index, edge_attr, batch)

    def decode(self, z, edge_index, batch=None):
        return self.decoder(z, edge_index, batch)

    def discriminate(self, z):
        return self.discriminator(z)

    def forward(self, x, edge_index, edge_attr, batch=None):
        z, mu = self.encode(x, edge_index, edge_attr, batch)
        reconstructed_x, reconstructed_edge_attr = self.decode(z, edge_index, batch)
        return reconstructed_x, reconstructed_edge_attr, z

    def generate_multiple_outputs(self, x, edge_index, edge_attr, batch=None, num_samples=5, variator=0.1):
        outputs = []
        with torch.no_grad():
            z, mu = self.encode(x, edge_index, edge_attr, batch)
            for _ in range(num_samples):
                epsilon = torch.randn_like(mu) * variator
                z_new = mu + epsilon
                reconstructed_x, reconstructed_edge_attr = self.decode(z_new, edge_index, batch)
                outputs.append((reconstructed_x, reconstructed_edge_attr, edge_index))
        return outputs

class Encoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim, dropout):
        super(Encoder, self).__init__()
        self.edge_encoder = nn.Linear(edge_features, hidden_dim)
        self.conv1 = SAGEConv(node_features + hidden_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.layer_norm1 = nn.LayerNorm(hidden_dim)
        self.layer_norm2 = nn.LayerNorm(hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_var = nn.Linear(hidden_dim, latent_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index, edge_attr, batch=None):
        edge_attr = torch.clamp(edge_attr, -5, 5)
        edge_attr = F.relu(self.edge_encoder(edge_attr))
        edge_attr = self.dropout(edge_attr)
        
        edge_attr_expanded = torch.zeros((x.size(0), edge_attr.size(1)), device=x.device)
        edge_attr_expanded[edge_index[0]] = edge_attr
        
        x = torch.cat([x, edge_attr_expanded], dim=1)
        x = torch.clamp(x, -5, 5)
        
        h1 = self.conv1(x, edge_index)
        h1 = self.layer_norm1(h1)
        h1 = F.relu(h1)
        h1 = self.dropout(h1)
        
        h2 = self.conv2(h1, edge_index)
        h2 = self.layer_norm2(h2)
        h2 = F.relu(h2)
        h2 = self.dropout(h2)
        
        if batch is not None:
            h2 = global_mean_pool(h2, batch)
        
        mu = self.fc_mu(h2)
        log_var = self.fc_var(h2)
        log_var = torch.clamp(log_var, -10, 2)
        
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        z = mu + eps * std
        
        return z, mu

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, node_features, edge_features):
        super(Decoder, self).__init__()
        self.node_decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, node_features)
        )
        
        self.edge_decoder = nn.Sequential(
            nn.Linear(latent_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, edge_features)
        )

    def forward(self, z, edge_index, batch=None):
        if batch is not None:
            z = z[batch]
        
        reconstructed_x = self.node_decoder(z)
        
        row, col = edge_index
        edge_h = torch.cat([z[row], z[col]], dim=1)
        reconstructed_edge_attr = self.edge_decoder(edge_h)
        
        return reconstructed_x, reconstructed_edge_attr

class Discriminator(nn.Module):
    def __init__(self, latent_dim, hidden_dim):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, z):
        return self.net(z)

In [5]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader as PyGDataLoader
def train_arga_with_early_stopping(model, train_loader, val_loader, num_epochs, lr=0.0001, device='cuda', patience=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            
            reconstructed_x, reconstructed_edge_attr, z = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            
            recon_loss = F.mse_loss(reconstructed_x, batch.x) + F.mse_loss(reconstructed_edge_attr, batch.edge_attr)
          
            real_z = torch.randn_like(z)
            real_scores = model.discriminate(real_z)
            fake_scores = model.discriminate(z)
            
            adv_loss = -torch.mean(torch.log(torch.clamp(fake_scores, 1e-6, 1.0)) + 
                                 torch.log(torch.clamp(1 - real_scores, 1e-6, 1.0)))
            
            loss = recon_loss + 1 * adv_loss
            
            if not torch.isnan(loss) and not torch.isinf(loss):
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                total_loss += loss.item()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                reconstructed_x, reconstructed_edge_attr, _ = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
                val_loss += (F.mse_loss(reconstructed_x, batch.x) + 
                           F.mse_loss(reconstructed_edge_attr, batch.edge_attr)).item()
        
        val_loss /= len(val_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print('Early stopping triggered')
                break
    
    return model


In [6]:
import optuna
import torch.optim as optim
import random
import multiprocessing

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def objective(trial):
    """
    Objective function for hyperparameter optimization.
    """
    # Define hyperparameters to search
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512, step=64)
    latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    patience = trial.suggest_int("patience", 3, 10)

    # Print the current trial's hyperparameters
    print(f"\n[Trial {trial.number}] Starting with parameters: {trial.params}")

    # Prepare dataset loaders
    pyg_dataset = torch.load('../../data/curated dataset/cleaned_frag_pyg_dataset.pth')
    train_ratio = 0.8
    num_train = int(len(pyg_dataset) * train_ratio)
    num_valid = len(pyg_dataset) - num_train

    train_dataset, valid_dataset = random_split(pyg_dataset, [num_train, num_valid])
    train_loader = PyGDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = PyGDataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    node_features = pyg_dataset[0].x.size(1)
    edge_features = pyg_dataset[0].edge_attr.size(1)
    model = ARGA(node_features, edge_features, hidden_dim, latent_dim)

    # Train the model
    trained_model = train_arga_with_early_stopping(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=60,
        lr=lr,
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        patience=patience
    )

    # Calculate validation loss or other metrics
    model.eval()

    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            reconstructed_x, reconstructed_edge_attr, _ = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            val_loss += (F.mse_loss(reconstructed_x, batch.x) + 
                        F.mse_loss(reconstructed_edge_attr, batch.edge_attr)).item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"[Trial {trial.number}] Validation Loss: {avg_val_loss:.4f}")

    # Return validation loss as the optimization metric
    return avg_val_loss


# Define and run the Optuna study
if __name__ == "__main__":
    num_cores = multiprocessing.cpu_count()  # Use all available cores
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=200, n_jobs=num_cores)  # Set n_jobs to the number of CPU cores

    print("Best hyperparameters:", study.best_params)

[I 2025-01-08 10:10:14,546] A new study created in memory with name: no-name-1fc3b48d-ab78-4b60-9d34-6eae675e279a
C:\Users\sarab\AppData\Local\Temp\ipykernel_25536\533947880.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub f


[Trial 1] Starting with parameters: {'hidden_dim': 256, 'latent_dim': 32, 'lr': 0.0005670950504210985, 'batch_size': 16, 'patience': 4}
[Trial 0] Starting with parameters: {'hidden_dim': 512, 'latent_dim': 64, 'lr': 0.04448765643911486, 'batch_size': 64, 'patience': 10}


[Trial 2] Starting with parameters: {'hidden_dim': 256, 'latent_dim': 64, 'lr': 0.001368526281729318, 'batch_size': 8, 'patience': 8}

[Trial 3] Starting with parameters: {'hidden_dim': 384, 'latent_dim': 32, 'lr': 1.9255673577850748e-05, 'batch_size': 64, 'patience': 10}

[Trial 4] Starting with parameters: {'hidden_dim': 512, 'latent_dim': 96, 'lr': 0.0013606265013507803, 'batch_size': 64, 'patience': 4}

[Trial 5] Starting with parameters: {'hidden_dim': 192, 'latent_dim': 64, 'lr': 2.2632688109496862e-05, 'batch_size': 16, 'patience': 6}

[Trial 6] Starting with parameters: {'hidden_dim': 256, 'latent_dim': 128, 'lr': 0.0011861753762190549, 'batch_size': 8, 'patience': 6}

[Trial 8] Starting with parameters: {'hi

[I 2025-01-08 10:17:29,750] Trial 12 finished with value: 0.08921072880427043 and parameters: {'hidden_dim': 256, 'latent_dim': 128, 'lr': 0.001148154498740274, 'batch_size': 64, 'patience': 3}. Best is trial 12 with value: 0.08921072880427043.



[Trial 16] Starting with parameters: {'hidden_dim': 128, 'latent_dim': 96, 'lr': 0.035810794129719443, 'batch_size': 64, 'patience': 4}
Epoch 12/60, Loss: 0.3119, Val Loss: 0.0966
Epoch 4/60, Loss: 0.0891, Val Loss: 0.0924
Epoch 8/60, Loss: 0.0896, Val Loss: 0.0877
Epoch 12/60, Loss: 0.1094, Val Loss: 0.0945
Epoch 12/60, Loss: 0.1423, Val Loss: 0.0930
Epoch 12/60, Loss: 0.0891, Val Loss: 0.0884
Epoch 12/60, Loss: 0.0887, Val Loss: 0.0893
Epoch 12/60, Loss: 0.0997, Val Loss: 0.1006
Epoch 13/60, Loss: 0.2802, Val Loss: 0.0956
Epoch 13/60, Loss: 0.0929, Val Loss: 0.0910
Epoch 8/60, Loss: 0.0946, Val Loss: 0.0899
Epoch 1/60, Loss: 0.3524, Val Loss: 0.1272
Epoch 9/60, Loss: 0.0896, Val Loss: 0.0878
Epoch 13/60, Loss: 0.1361, Val Loss: 0.0926
Epoch 13/60, Loss: 0.0891, Val Loss: 0.0884
Epoch 14/60, Loss: 0.2556, Val Loss: 0.0950
Epoch 13/60, Loss: 0.0886, Val Loss: 0.0892
Epoch 13/60, Loss: 0.0996, Val Loss: 0.1009
Epoch 5/60, Loss: 0.2678, Val Loss: 0.1007
Epoch 14/60, Loss: 0.0912, Val Lo

[I 2025-01-08 10:23:54,038] Trial 16 finished with value: 0.09885171006123225 and parameters: {'hidden_dim': 128, 'latent_dim': 96, 'lr': 0.035810794129719443, 'batch_size': 64, 'patience': 4}. Best is trial 12 with value: 0.08921072880427043.


...

[Trial 191] Validation Loss: 0.0871
Epoch 49/60, Loss: 0.0880, Val Loss: 0.0876
Epoch 48/60, Loss: 0.0879, Val Loss: 0.0871
Epoch 50/60, Loss: 0.0879, Val Loss: 0.0876
Epoch 51/60, Loss: 0.0878, Val Loss: 0.0877
Epoch 49/60, Loss: 0.0878, Val Loss: 0.0868
Epoch 52/60, Loss: 0.0877, Val Loss: 0.0876
Epoch 50/60, Loss: 0.0882, Val Loss: 0.0868
Epoch 53/60, Loss: 0.0879, Val Loss: 0.0877
Epoch 54/60, Loss: 0.0878, Val Loss: 0.0876
Epoch 51/60, Loss: 0.0880, Val Loss: 0.0869
Epoch 55/60, Loss: 0.0879, Val Loss: 0.0876
Epoch 56/60, Loss: 0.0882, Val Loss: 0.0876
Epoch 52/60, Loss: 0.0881, Val Loss: 0.0868
Epoch 57/60, Loss: 0.0881, Val Loss: 0.0877
Epoch 58/60, Loss: 0.0877, Val Loss: 0.0878
Epoch 53/60, Loss: 0.0888, Val Loss: 0.0870
Epoch 59/60, Loss: 0.0877, Val Loss: 0.0876
Epoch 60/60, Loss: 0.0881, Val Loss: 0.0877


[I 2025-01-08 22:53:28,275] Trial 198 finished with value: 0.08767610726257165 and parameters: {'hidden_dim': 128, 'latent_dim': 128, 'lr': 0.00023080480199399217, 'batch_size': 16, 'patience': 6}. Best is trial 122 with value: 0.08478449912120899.


[Trial 198] Validation Loss: 0.0877
Epoch 54/60, Loss: 0.0880, Val Loss: 0.0868
Epoch 55/60, Loss: 0.0878, Val Loss: 0.0868
Epoch 56/60, Loss: 0.0879, Val Loss: 0.0870
Epoch 57/60, Loss: 0.0879, Val Loss: 0.0869
Epoch 58/60, Loss: 0.0878, Val Loss: 0.0868
Epoch 59/60, Loss: 0.0880, Val Loss: 0.0870
Epoch 60/60, Loss: 0.0878, Val Loss: 0.0868


[I 2025-01-08 22:54:44,971] Trial 194 finished with value: 0.08682398398717245 and parameters: {'hidden_dim': 256, 'latent_dim': 128, 'lr': 0.00023764721058076808, 'batch_size': 16, 'patience': 6}. Best is trial 122 with value: 0.08478449912120899.


[Trial 194] Validation Loss: 0.0868
Best hyperparameters: {'hidden_dim': 192, 'latent_dim': 64, 'lr': 0.0012589907254044416, 'batch_size': 8, 'patience': 10}
